# 1.6 Cryptoanalysis of Monoalphabetic Substitution Ciphers

In [1]:
%%capture
%run ./1_5_simple_ciphers.ipynb

In [2]:
from src.helpers import display_fequency_tables, reverse_mult_key, reverse_add_key, mod_26, build_guesses

## Exercises 1.6


Some lists and utility classes to simplify the ciphertext cracking below.  Helper functions are in [helpers.py](https://github.com/dandoug/cryptomath-book/blob/main/src/helpers.py).

In [3]:
# top occurring letters and digraphs in English plaintext, used to construct guesses below
top_digraphs = ['th', 'er', 'on', 'an', 're', 'he', 'in', 'ed', 'en', 'nd']
top_letters = ['e', 't', 'a', 'o', 'i', 'n', 's', 'h']

In [4]:
class CrackAdditiveCipher(AdditiveCipher):
    """
    A CrackAdditiveCipher is a subclass of AdditiveCipher that takes a tuple of two
    characters as the "key".  These are assumed to be a plaintext letter and the
    corresponding ciphertext character, example ('e', 'Z')
    In an AdditiveCipher, the key can be computed from that information.
    """
    def __init__(self, tuple_guess: tuple[str, str]):
        add_key = reverse_add_key(plain_char=tuple_guess[0], cipher_char=tuple_guess[1])
        super().__init__(add_key=add_key)

    def key(self) -> int:
        return self._add_key


In [5]:
class CrackMultiplicativeCipher(MultiplicativeCipher):
    """
    A CrackMultiplicativeCipher is a subclass of MultiplicativeCipher that takes a tuple of two
    characters as the "key".  These are assumed to be a plaintext letter and the
    corresponding ciphertext character, example ('e', 'Z')
    In an MultiplicativeCipher, the key can be computed from that information.
    """
    def __init__(self, tuple_guess: tuple[str, str]):
        """
        :raise ValueError if no key can be found.
        """
        mult_key = reverse_mult_key(plain_char=tuple_guess[0], cipher_char=tuple_guess[1])
        super().__init__(mult_key=mult_key)

    def key(self) -> int:
        return self._mult_key

In [6]:
class CrackAffineCipher(AffineCipher):
    """
    A CrackAffineCipher is a subclass of AffineCipher that takes two tuples of (plaintext letter, ciphertext letter) pairs
    and solves for the multiplicative and additive keys necessary to produce the given characters if a solution exists.
    """
    def __init__(self, tuple_guesses: tuple[tuple[str, str], tuple[str, str]]):
        """
        :param tuple_guesses: example (('e', 'Z'), ('t', 'J'))
        :raise ValueError if no keys can be found for the given guesses.
        """
        p1 = pos(tuple_guesses[0][0])
        c1 = pos(tuple_guesses[0][1])
        p2 = pos(tuple_guesses[1][0])
        c2 = pos(tuple_guesses[1][1])
        mult_key = mod_26((c1 - c2) * inv_26(mod_26(p1 - p2)))
        add_key = mod_26((c1 - mult_key * p1) * inv_26(mult_key))
        super().__init__(mult_key=mult_key, add_key=add_key)

    def mult_key(self) -> int:
        return self._mult_key
    def add_key(self) -> int:
        return self._add_key

Question 7. done first so that the frequency analsysis program will be available to use with the other problems.


### 7. Write a program that accepts a text message and outputs the frequency of each letter appearing in the message.

In [7]:
def frequency_tabulator(message: str) -> tuple[ dict[str, int], dict[str, int], dict[str, int] ]:
    """
    Given a message, return the feequency of each letter appearing in the message, each
    diagraph and each trigraph.
    """
    letter_counts: dict[str, int] = {}
    diagram_counts: dict[str, int] = {}
    trigraph_counts: dict[str, int] = {}
    current_digraph = current_trigraph = ''
    message = strip_text(message)
    for c in message:
        current_letter = c
        current_digraph = (current_digraph + c)[-2:]
        current_trigraph = (current_trigraph + c)[-3:]
        letter_counts[current_letter] = letter_counts.get(current_letter, 0) + 1
        if len(current_digraph) == 2:
            diagram_counts[current_digraph] = diagram_counts.get(current_digraph, 0) + 1
        if len(current_trigraph) == 3:
            trigraph_counts[current_trigraph] = trigraph_counts.get(current_trigraph, 0) + 1
    return (
        dict(sorted(letter_counts.items(), key=lambda item: item[1], reverse=True)),
        dict(sorted(diagram_counts.items(), key=lambda item: item[1], reverse=True)),
        dict(sorted(trigraph_counts.items(), key=lambda item: item[1], reverse=True)))

### 1. Decipher the following message.

In [8]:
msg1 = (
    "KFM YGV VEM VHK AWK YZK FWG RKF MSJ JZG XOJ MEM DJZ MAM SCJ " +
    "GKF EJK TSF GJI STM ZSW MKF MEJ KBS XGJ SFH PMJ JIK FME JKR MSZ"
)

In [9]:
letters, digraphs, trigraphs = frequency_tabulator(msg1)
display_fequency_tables(letters, digraphs, trigraphs)

<IPython.core.display.Math object>

First, try the top additive ciphers that might make sense by taking to top 4 occurring ciphertext letters and pairing them with the top occurring plaintext letters.

In [10]:
# Build a set of single letter guesses and the top 4 occurring ciphertext letters
guesses = build_guesses(top_letters, list(letters.keys())[:4])

In [11]:
for guess in guesses:
    crack = CrackAdditiveCipher(guess)
    print(f"\nTrying an additive cipher derived from guess={guess} with key={crack.key()}")
    plaintext = crack.decrypt(msg1)
    print(format_plaintext(plaintext))


Trying an additive cipher derived from guess=('e', 'M') with key=8
cxeqynnwenzcsocqrcxoyjcxekbbrypgbewevbresekubycxwbclkxybakle
rkoecxewbctkpybkxzhebbacxewbcjekr

Trying an additive cipher derived from guess=('e', 'J') with key=5
fahtbqqzhqcfvrftufarbmfahneeubsjehzhyeuhvhnxebfazefonabednoh
unrhfahzefwnsbenackheedfahzefmhnu

Trying an additive cipher derived from guess=('e', 'K') with key=6
ezgsappygpbeuqestezqalezgmddtaridgygxdtgugmwdaezydenmzadcmng
tmqgezgydevmradmzbjgddcezgydelgmt

Trying an additive cipher derived from guess=('e', 'F') with key=1
jelxfuudlugjzvjxyjevfqjelriiyfwnildlciylzlrbifjedijsrefihrsl
yrvljeldijarwfiregoliihjeldijqlry

Trying an additive cipher derived from guess=('t', 'M') with key=19
rmtfnccltcorhdrfgrmdnyrmtzqqgnevqtltkqgthtzjqnrmlqrazmnqpzat
gzdtrmtlqrizenqzmowtqqprmtlqrytzg

Trying an additive cipher derived from guess=('t', 'J') with key=16
upwiqffowfrukguijupgqbupwcttjqhytwowntjwkwcmtqupotudcpqtscdw
jcgwupwotulchqtcprzwttsupwotubwcj

Trying an additive 

Those didn't work.  Let's see if any of those guesses make a good multiplicative cipher.

In [12]:
for guess in guesses:
    try:
        crack = CrackMultiplicativeCipher(guess)
        print(f"\nTrying an multiplicative cipher derived from guess={guess} with key={crack.key()}")
        plaintext = crack.decrypt(msg1)
        print(format_plaintext(plaintext))
    except ValueError:
        print(f"\nGuess: {guess} is not valid for multiplicative key")


Guess: ('e', 'M') is not valid for multiplicative key

Guess: ('e', 'J') is not valid for multiplicative key

Trying an multiplicative cipher derived from guess=('e', 'K') with key=23
exmiojjgmjfeqaeizexaotexmknnzorunmgmpnzmqmkynoexgnebkxonwkbm
zkamexmgnehkronkxflmnnwexmgnetmkz

Guess: ('e', 'F') is not valid for multiplicative key

Guess: ('t', 'M') is not valid for multiplicative key

Trying an multiplicative cipher derived from guess=('t', 'J') with key=7
ilmkarrwmrpiogikzilgajilmyttzavqtmwmhtzmomystailwtinylateynm
zygmilmwtidyvatylpfmtteilmwtijmyz

Guess: ('t', 'K') is not valid for multiplicative key

Trying an multiplicative cipher derived from guess=('t', 'F') with key=25
otmasddumdroycoazotcshotmgppzsbkpmumvpzmymgwpsotupofgtspqgfm
zgcmotmupoxgbspgtrjmppqotmupohmgz

Guess: ('a', 'M') is not valid for multiplicative key

Guess: ('a', 'J') is not valid for multiplicative key

Trying an multiplicative cipher derived from guess=('a', 'K') with key=11
ajmgcbbqmbvasuagzajucdajmwhhzcn

So, none of those choices for additive or multiplicative ciphers yielded results.  Let's try affine ciphers with the top digraphs.

In [13]:
guesses = build_guesses(top_digraphs, list(digraphs.keys())[:4])
for guess in guesses:
    try:
        crack = CrackAffineCipher(guess)
        print(f"\nTrying possible solution {guess}: mult_key:{crack.mult_key()}  add_key:{crack.add_key()}")
        plaintext = crack.decrypt(msg1)
        print(format_plaintext(plaintext))
    except ValueError:
        print(f"\n{guess} not a possible solution")



(('t', 'K'), ('h', 'F')) not a possible solution

(('t', 'F'), ('h', 'M')) not a possible solution

(('t', 'M'), ('h', 'S')) not a possible solution

(('t', 'M'), ('h', 'E')) not a possible solution

(('e', 'K'), ('r', 'F')) not a possible solution

(('e', 'F'), ('r', 'M')) not a possible solution

(('e', 'M'), ('r', 'S')) not a possible solution

(('e', 'M'), ('r', 'E')) not a possible solution

Trying possible solution (('o', 'K'), ('n', 'F')): mult_key:5  add_key:8
onewillseldomgowrongifoneattributesextremeactionstovanityave
rageonestohabitandpettyonestofear

Trying possible solution (('o', 'F'), ('n', 'M')): mult_key:19  add_key:25
ronpziidnikrltrparotzqronbggazejgndnsganlnbhgzrodgrmbozgvbmn
abtnrondgrwbezgbokunggvrondgrqnba

(('o', 'M'), ('n', 'S')) not a possible solution

(('o', 'M'), ('n', 'E')) not a possible solution

(('a', 'K'), ('n', 'F')) not a possible solution

(('a', 'F'), ('n', 'M')) not a possible solution

(('a', 'M'), ('n', 'S')) not a possible solution

(('a', 'M

An **AffineCipher** using guess `(('o', 'K'), ('n', 'F')): mult_key:5  add_key:8` yielded a good result:
```
one will seldom go wrong if one attributes extreme actions to vanity average
ones to habit and petty ones to fear
```


### 2. Decipher the following message.

In [14]:
msg2 = (
    "XTS OCZ SIV JPI FCS XQE BEA SIV YIP SOF ICO SJR QYC VJJ SNS VJE VXT " +
    "FSS BIA XEF OXT SPS DEA CXQ EBX TSY CVJ XTS JCO XIV ASC XRD EYO IAF " +
    "EOO XTS YIX SFX TSD SVK XTE BXC MSC XRD EYO"
)

In [15]:
letters, digraphs, trigraphs = frequency_tabulator(msg2)
display_fequency_tables(letters, digraphs, trigraphs)

<IPython.core.display.Math object>

In [16]:
# Build a set of single letter guesses and the top 4 occurring ciphertext letters
guesses = build_guesses(top_letters, list(letters.keys())[:4])

In [17]:
for guess in guesses:
    crack = CrackAdditiveCipher(guess)
    print(f"\nTrying an additive cipher derived from guess={guess} with key={crack.key()}")
    plaintext = crack.decrypt(msg2)
    print(format_plaintext(plaintext))


Trying an additive cipher derived from guess=('e', 'S') with key=14
jfeaoleuhvburoejcqnqmeuhkubearuoaevdckohvvezehvqhjfreenumjqr
ajfebepqmojcqnjfekohvjfevoajuhmeojdpqkaumrqaajfekujerjfepehw
jfqnjoyeojdpqka

Trying an additive cipher derived from guess=('e', 'X') with key=19
eazvjgzpcqwpmjzexlilhzpcfpwzvmpjvzqyxfjcqqzuzcqlceamzziphelm
veazwzklhjexlieazfjcqeazqjvepchzjeyklfvphmlvveazfpezmeazkzcr
ealiejtzjeyklfv

Trying an additive cipher derived from guess=('e', 'C') with key=24
zvuqebukxlrkheuzsgdgcukxakruqhkequltsaexllupuxlgxzvhuudkczgh
qzvurufgcezsgdzvuaexlzvuleqzkxcueztfgaqkchgqqzvuakzuhzvufuxm
zvgdzeoueztfgaq

Trying an additive cipher derived from guess=('e', 'E') with key=0
xtsoczsivjpifcsxqebeasivyipsoficosjrqycvjjsnsvjevxtfssbiaxef
oxtspsdeacxqebxtsycvjxtsjcoxivascxrdeyoiafeooxtsyixsfxtsdsvk
xtebxcmscxrdeyo

Trying an additive cipher derived from guess=('t', 'S') with key=25
yutpdatjwkqjgdtyrfcfbtjwzjqtpgjdptksrzdwkktotwkfwyugttcjbyfg
pyutqtefbdyrfcyutzdwkyutkdpyjwbtdysefzpjbgf

In [18]:
for guess in guesses:
    try:
        crack = CrackMultiplicativeCipher(guess)
        print(f"\nTrying an multiplicative cipher derived from guess={guess} with key={crack.key()}")
        plaintext = crack.decrypt(msg2)
        print(format_plaintext(plaintext))
    except ValueError:
        print(f"\nGuess: {guess} is not valid for multiplicative key")


Trying an multiplicative cipher derived from guess=('e', 'S') with key=9
thesizeandvarietyofoceanwavesraisedbywinddependonthreefactor
sthevelocityofthewindthedistanceitblowsacrossthewatertheleng
thoftimeitblows

Guess: ('e', 'X') is not valid for multiplicative key

Trying an multiplicative cipher derived from guess=('e', 'C') with key=11
npwyezwobhrojewnkqlqswobgorwyjoeywhdkgebhhwfwbhqbnpjwwlosnqj
ynpwrwxqsenkqlnpwgebhnpwheynobswendxqgyosjqyynpwgonwjnpwxwba
npqlnemwendxqgy

Trying an multiplicative cipher derived from guess=('e', 'E') with key=1
xtsoczsivjpifcsxqebeasivyipsoficosjrqycvjjsnsvjevxtfssbiaxef
oxtspsdeacxqebxtsycvjxtsjcoxivascxrdeyoiafeooxtsyixsfxtsdsvk
xtebxcmscxrdeyo

Guess: ('t', 'S') is not valid for multiplicative key

Trying an multiplicative cipher derived from guess=('t', 'X') with key=9
thesizeandvarietyofoceanwavesraisedbywinddependonthreefactor
sthevelocityofthewindthedistanceitblowsacrossthewatertheleng
thoftimeitblows

Guess: ('t', 'C') is not valid for multi

A **MultiplicativeCipher** derived from `guess=('e', 'S') with key=9` yielded
```
the size and variety of ocean waves raised by wind depend on three factors
the velocity of the wind the distance it blows across the water the length
of time it blows
```

### 3. Decipher the following message.

In [19]:
msg3 = (
    "VYU XOX SJY YTS FXV OWM YFQ GCQ PPY CQP VQD OFP VQJ ODW PRT SEQ"
)

In [20]:
letters, digraphs, trigraphs = frequency_tabulator(msg3)
display_fequency_tables(letters, digraphs, trigraphs)

<IPython.core.display.Math object>

In [21]:
# Build a set of single letter guesses and the top 4 occurring ciphertext letters
guesses = build_guesses(top_letters, list(letters.keys())[:4])

In [22]:
for guess in guesses:
    crack = CrackAdditiveCipher(guess)
    print(f"\nTrying an additive cipher derived from guess={guess} with key={crack.key()}")
    plaintext = crack.decrypt(msg3)
    print(format_plaintext(plaintext))


Trying an additive cipher derived from guess=('e', 'Q') with key=12
jmilclgxmmhgtljckamteuqeddmqedjerctdjexcrkdfhgse

Trying an additive cipher derived from guess=('e', 'Y') with key=20
beadudypeezyldbucselwmiwvveiwvbwjulvbwpujcvxzykw

Trying an additive cipher derived from guess=('e', 'P') with key=11
knjmdmhynnihumkdlbnufvrfeenrfekfsduekfydslegihtf

Trying an additive cipher derived from guess=('e', 'V') with key=17
ehdgxgbshhcbogexfvhozplzyyhlzyezmxoyezsxmfyacbnz

Trying an additive cipher derived from guess=('t', 'Q') with key=23
ybxaravmbbwviayrzpbitjftssbftsytgrisytmrgzsuwvht

Trying an additive cipher derived from guess=('t', 'Y') with key=5
qtpsjsnettonasqjrhtalbxlkktxlkqlyjakqlejyrkmonzl

Trying an additive cipher derived from guess=('t', 'P') with key=22
zcybsbwnccxwjbzsaqcjukguttcgutzuhsjtzunshatvxwiu

Trying an additive cipher derived from guess=('t', 'V') with key=2
twsvmvqhwwrqdvtmukwdoeaonnwaontobmdntohmbunprqco

Trying an additive cipher derived from guess=('a', 'Q') w

In [23]:
for guess in guesses:
    try:
        crack = CrackMultiplicativeCipher(guess)
        print(f"\nTrying an multiplicative cipher derived from guess={guess} with key={crack.key()}")
        plaintext = crack.decrypt(msg3)
        print(format_plaintext(plaintext))
    except ValueError:
        print(f"\nGuess: {guess} is not valid for multiplicative key")


Trying an multiplicative cipher derived from guess=('e', 'Q') with key=19
howdidafoolandhismoneygettogetherinthefirstplace

Trying an multiplicative cipher derived from guess=('e', 'Y') with key=5
teyjcjibeedivjtcomevsqksxxeksxtsfcvxtsbcfoxndias

Guess: ('e', 'P') is not valid for multiplicative key

Guess: ('e', 'V') is not valid for multiplicative key

Guess: ('t', 'Q') is not valid for multiplicative key

Guess: ('t', 'Y') is not valid for multiplicative key

Trying an multiplicative cipher derived from guess=('t', 'P') with key=19
howdidafoolandhismoneygettogetherinthefirstplace

Trying an multiplicative cipher derived from guess=('t', 'V') with key=5
teyjcjibeedivjtcomevsqksxxeksxtsfcvxtsbcfoxndias

Trying an multiplicative cipher derived from guess=('a', 'Q') with key=17
lcofgfuvccruhflgimchaeqaddcqadlanghdlavgnidxruka

Trying an multiplicative cipher derived from guess=('a', 'Y') with key=25
daebkbgpaafgtbdkcmatiswijjawijdivktjdipkvcjhfgui

Guess: ('a', 'P') is not valid for mu

A **MultiplicativeCipher** derived from `guess=('e', 'Q') with key=19` yielded

```
how did a fool and his money get together in the first place
```

### 4. Decipher the following message.

In [24]:
msg4 = (
    "STB RDX TZQ MFY MJQ GTB WTT RXM FPJ XUJ FWJ MFI PNS LOT MSX FDF " +
    "UUF WJS YQD MJB FXS YTS YMJ BFX MNS LYT SGJ QYB FDF YYM JYN RJ"
)

In [25]:
letters, digraphs, trigraphs = frequency_tabulator(msg4)
display_fequency_tables(letters, digraphs, trigraphs)

<IPython.core.display.Math object>

In [26]:
# Build a set of single letter guesses and the top 4 occurring ciphertext letters
guesses = build_guesses(top_letters, list(letters.keys())[:4])

In [27]:
for guess in guesses:
    crack = CrackAdditiveCipher(guess)
    print(f"\nTrying an additive cipher derived from guess={guess} with key={crack.key()}")
    plaintext = crack.decrypt(msg4)
    print(format_plaintext(plaintext))


Trying an additive cipher derived from guess=('e', 'F') with key=1
rsaqcwsyplexlipfsavssqwleoiwtievilehomrknslrwecettevirxpclia
ewrxsrxliaewlmrkxsrfipxaecexxlixmqi

Trying an additive cipher derived from guess=('e', 'J') with key=5
nowmysoulhathelbowroomshakespearehadkingjohnsayapparentlyhew
asntonthewashingtonbeltwayatthetime

Trying an additive cipher derived from guess=('e', 'M') with key=8
kltjvplriexqebiyltolljpexhbpmbxobexahfkdglekpxvxmmxobkqivebt
xpkqlkqebtxpefkdqlkybiqtxvxqqebqfjb

Trying an additive cipher derived from guess=('e', 'Y') with key=20
yzhxjdzfwslespwmzhczzxdslvpdaplcpslovtyruzsydljlaalcpyewjsph
ldyezyesphldstyrezympwehljleespetxp

Trying an additive cipher derived from guess=('t', 'F') with key=12
ghpfrlhneatmaxeuhpkhhflatdxlixtkxatwdbgzchagltrtiitkxgmeraxp
tlgmhgmaxptlabgzmhguxemptrtmmaxmbfx

Trying an additive cipher derived from guess=('t', 'J') with key=16
cdlbnhdjawpiwtaqdlgddbhwpzthetpgtwpszxcvydwchpnpeepgtcianwtl
phcidciwtlphwxcvidcqtailpnpiiwtixbt

Trying

An *AdditiveCipher* derived from `guess=('e', 'J')` with `key=5` yielded

```
now my soul hath elbow room shakespeare had king john say apparently he wasnt
on the washington beltway at the time
```

### 5. Decipher the following message.

In [28]:
msg5 = (
    "CWF FSK AAC KHW JAO ZHA NGA THO ZUW ENC AKK MUA KJA HR"
)

In [29]:
letters, digraphs, trigraphs = frequency_tabulator(msg5)
display_fequency_tables(letters, digraphs, trigraphs)

<IPython.core.display.Math object>

In [30]:
# Build a set of single letter guesses and the top 4 occurring ciphertext letters
guesses = build_guesses(top_letters, list(letters.keys())[:4])

In [31]:
for guess in guesses:
    crack = CrackAdditiveCipher(guess)
    print(f"\nTrying an additive cipher derived from guess={guess} with key={crack.key()}")
    plaintext = crack.decrypt(msg5)
    print(format_plaintext(plaintext))


Trying an additive cipher derived from guess=('e', 'A') with key=22
gajjwoeegolanesdlerkexlsdyairgeooqyeonelv

Trying an additive cipher derived from guess=('e', 'K') with key=6
wqzzmeuuwebqduitbuhaunbitoqyhwueegouedubl

Trying an additive cipher derived from guess=('e', 'H') with key=3
ztccphxxzhetgxlwexkdxqelwrtbkzxhhjrxhgxeo

Trying an additive cipher derived from guess=('e', 'C') with key=24
eyhhumccemjylcqbjcpicvjqbwygpecmmowcmlcjt

Trying an additive cipher derived from guess=('t', 'A') with key=7
vpyyldttvdapcthsatgztmahsnpxgvtddfntdctak

Trying an additive cipher derived from guess=('t', 'K') with key=17
lfoobtjjltqfsjxiqjwpjcqxidfnwljttvdjtsjqa

Trying an additive cipher derived from guess=('t', 'H') with key=14
oirrewmmowtivmaltmzsmftalgiqzomwwygmwvmtd

Trying an additive cipher derived from guess=('t', 'C') with key=9
tnwwjbrrtbynarfqyrexrkyfqlnvetrbbdlrbaryi

Trying an additive cipher derived from guess=('a', 'A') with key=0
cwffskaackhwjaozhangathozuwencakkmuakjahr

Tryin

In [32]:
for guess in guesses:
    try:
        crack = CrackMultiplicativeCipher(guess)
        print(f"\nTrying an multiplicative cipher derived from guess={guess} with key={crack.key()}")
        plaintext = crack.decrypt(msg5)
        print(format_plaintext(plaintext))
    except ValueError:
        print(f"\nGuess: {guess} is not valid for multiplicative key")


Trying an multiplicative cipher derived from guess=('e', 'A') with key=21
okddqceeocnkxewznerievnwzakyroeccmaecxenl

Trying an multiplicative cipher derived from guess=('e', 'K') with key=23
yaxxkeqqyefanquzfqdoqbfuzsagdyqeemsqenqft

Guess: ('e', 'H') is not valid for multiplicative key

Trying an multiplicative cipher derived from guess=('e', 'C') with key=11
eujjwasseavuhsyzvsfcspvyziuqfesaamisahsvd

Guess: ('t', 'A') is not valid for multiplicative key

Guess: ('t', 'K') is not valid for multiplicative key

Trying an multiplicative cipher derived from guess=('t', 'H') with key=3
aybbouiiautylieztivkixtezgysvaiuumgiulitf

Guess: ('t', 'C') is not valid for multiplicative key

Trying an multiplicative cipher derived from guess=('a', 'A') with key=1
cwffskaackhwjaozhangathozuwencakkmuakjahr

Trying an multiplicative cipher derived from guess=('a', 'K') with key=11
eujjwasseavuhsyzvsfcspvyziuqfesaamisahsvd

Guess: ('a', 'H') is not valid for multiplicative key

Trying an multiplicative

In [33]:
guesses = build_guesses(top_digraphs, list(digraphs.keys())[:4])
for guess in guesses:
    try:
        crack = CrackAffineCipher(guess)
        print(f"\nTrying possible solution {guess}: mult_key:{crack.mult_key()}  add_key:{crack.add_key()}")
        plaintext = crack.decrypt(msg5)
        print(format_plaintext(plaintext))
    except ValueError:
        print(f"\n{guess} not a possible solution")


(('t', 'J'), ('h', 'A')) not a possible solution

(('t', 'O'), ('h', 'Z')) not a possible solution

(('t', 'A'), ('h', 'K')) not a possible solution

(('t', 'C'), ('h', 'W')) not a possible solution

(('e', 'J'), ('r', 'A')) not a possible solution

(('e', 'O'), ('r', 'Z')) not a possible solution

(('e', 'A'), ('r', 'K')) not a possible solution

(('e', 'C'), ('r', 'W')) not a possible solution

Trying possible solution (('o', 'J'), ('n', 'A')): mult_key:9  add_key:15
tbccprnntribondkinafnsidkvbzatnrrxvnronim

Trying possible solution (('o', 'O'), ('n', 'Z')): mult_key:15  add_key:12
isddqmuuimrsfuonruhkuxroneswhiummaeumfurj

(('o', 'A'), ('n', 'K')) not a possible solution

(('o', 'C'), ('n', 'W')) not a possible solution

(('a', 'J'), ('n', 'A')) not a possible solution

(('a', 'O'), ('n', 'Z')) not a possible solution

(('a', 'A'), ('n', 'K')) not a possible solution

(('a', 'C'), ('n', 'W')) not a possible solution

(('r', 'J'), ('e', 'A')) not a possible solution

(('r', 'O'), (

A **AffineCipher** derived from guess  `(('i', 'O'), ('n', 'Z'))` with  `mult_key:23  add_key:12` yields

```
molly seems to be intercepting our messages beth
```

### 6. Decipher the following message.

In [34]:
msg6 = (
    "ARO EXJ FPE LJB QLK RJB OLR PFK PQF QRQ FLK PLC EFD EBO IBX OKF KDF " +
    "KZI RAF KDA RHB XKA QEB RKF SBO PFQ VLC KLO QEZ XOL IFK XTE BOB QEB " +
    "VEX SBZ IXP PBP FKX AAF QFL KQL YXP HBQ YXI IQB XJP"
)

In [35]:
letters, digraphs, trigraphs = frequency_tabulator(msg6)
display_fequency_tables(letters, digraphs, trigraphs)

<IPython.core.display.Math object>

In [36]:
guesses = build_guesses(top_letters, list(letters.keys())[:4])

In [37]:
for guess in guesses:
    crack = CrackAdditiveCipher(guess)
    print(f"\nTrying an additive cipher derived from guess={guess} with key={crack.key()}")
    plaintext = crack.decrypt(msg6)
    print(format_plaintext(plaintext))


Trying an additive cipher derived from guess=('e', 'F') with key=1
zqndwieodkiapkjqiankqoejopepqpekjokbdecdanhawnjejcejyhqzejcz
qgawjzpdaqjeranoepukbjknpdywnkhejwsdanapdaudwrayhwooaoejwzze
pekjpkxwogapxwhhpawio

Trying an additive cipher derived from guess=('e', 'B') with key=23
durhamishometonumerousinstitutionsofhigherlearningincludingd
ukeandtheuniversityofnorthcarolinawheretheyhaveclassesinaddi
tiontobasketballteams

Trying an additive cipher derived from guess=('e', 'K') with key=6
uliyrdzjyfdvkfeldvifljzejkzklkzfejfwyzxyvicvriezexzetcluzexu
lbvreukyvlezmvijzkpfwefikytrifczernyvivkyvpyrmvtcrjjvjzeruuz
kzfekfsrjbvksrcckvrdj

Trying an additive cipher derived from guess=('e', 'Q') with key=12
ofcslxtdszxpezyfxpczfdtydetefetzydzqstrspcwplcytyrtynwfotyro
fvplyoespfytgpcdtejzqyzcesnlczwtylhspcpespjslgpnwlddpdtyloot
etzyezmldvpemlwweplxd

Trying an additive cipher derived from guess=('t', 'F') with key=12
ofcslxtdszxpezyfxpczfdtydetefetzydzqstrspcwplcytyrtynwfotyro
fvplyoespfytgpcdtejz

An **AddativeCipher** derived from guess `guess=('e', 'B')` with `key=23` yielded

```
durham is home to numerous institutions of higher learning including duke
and the university of north carolina where they have classes in addition
to basketball teams
```